In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [108]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate
)

In [3]:
model = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful chatbot capable of performing a variety of tasks on the data available"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

In [65]:
prompt.input_variables

['chat_history', 'input']

In [80]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [81]:
print(memory.load_memory_variables({}))

{'chat_history': []}


In [12]:
memory.save_context(
    {"input": "Hi, my name is John"},  # user message
    {"output": "Hey John, what's up? I'm an AI model called J."}  # AI response
)

In [13]:
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='Hi, my name is John', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey John, what's up? I'm an AI model called J.", additional_kwargs={}, response_metadata={})]}


In [14]:
memory.save_context(
    {"input": "I'm researching the different types of conversational memory."},  # user message
    {"output": "That's interesting, what are some examples?"}  # AI response
)

In [15]:
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='Hi, my name is John', additional_kwargs={}, response_metadata={}), AIMessage(content="Hey John, what's up? I'm an AI model called J.", additional_kwargs={}, response_metadata={}), HumanMessage(content="I'm researching the different types of conversational memory.", additional_kwargs={}, response_metadata={}), AIMessage(content="That's interesting, what are some examples?", additional_kwargs={}, response_metadata={})]}


In [66]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [67]:
print(memory.load_memory_variables({}))

{'chat_history': []}


In [82]:
# chain_input = RunnablePassthrough.assign(
#     **{
#         'chat_history': RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
#     }
# )
# closure around the memory
def get_prompt_input(inputs):
    chat_history = (RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")).invoke({})
    return {"input": inputs["input"], "chat_history": chat_history}

chain_input = RunnableLambda(get_prompt_input)
# chain_input = {
#     "input": lambda x: x["input"],
#     "chat_history": lambda _: (RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")).invoke({})
# }
# prompt has input variables: chat_history, input
chain = chain_input | prompt | model

In [83]:
input = 'Hi Im Juan'
inputs = {"input": input}
response = chain.invoke(inputs)
print(type(response))
print(response)

<class 'langchain_core.messages.ai.AIMessage'>
content='Hello Juan! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 31, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BES14PDH8Jmh1y6e1RXT7Mjhshjio', 'finish_reason': 'stop', 'logprobs': None} id='run-8c32eee9-3908-472e-ad70-915522537e48-0' usage_metadata={'input_tokens': 31, 'output_tokens': 11, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [54]:
print(type(memory))
print(type(memory.chat_memory))
print(memory.load_memory_variables({}))

<class 'langchain.memory.buffer.ConversationBufferMemory'>
<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
{'chat_history': []}


In [84]:
memory.chat_memory.add_user_message(input)
memory.chat_memory.add_ai_message(response.content)
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content='Hi Im Juan', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Juan! How can I assist you today?', additional_kwargs={}, response_metadata={})]}


In [85]:
input = 'what is my name'
inputs = {"input": input}
response = chain.invoke(inputs)
print(response)

content='Your name is Juan. How can I help you, Juan?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 53, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BES1t5EozO1ZHf5MnChpPTkgCybX0', 'finish_reason': 'stop', 'logprobs': None} id='run-af08d158-9923-4f7d-9778-c1b63de5aa4a-0' usage_metadata={'input_tokens': 53, 'output_tokens': 14, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [87]:
pipeline = prompt | model

In [88]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_map = {}
def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [89]:
from langchain_core.runnables.history import RunnableWithMessageHistory

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

In [90]:
pipeline_with_history.invoke(
    {"input": "Hi, my name is Juan"},
    config={"session_id": "id_123"}
)

AIMessage(content='Hello Juan! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 34, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BESGbTPvh70o4vqri7O5bIsECfbmv', 'finish_reason': 'stop', 'logprobs': None}, id='run-728ff3d8-9bbf-4aa1-bd2e-ac932b0a8398-0', usage_metadata={'input_tokens': 34, 'output_tokens': 11, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [92]:
pipeline_with_history.invoke(
    {"input": "What is my name again?"},
    config={"session_id": "id_123"}
)

AIMessage(content='Your name is Juan. How can I help you, Juan?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 58, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BESHFlachkdGbJeuhhxSLBnSZmxDq', 'finish_reason': 'stop', 'logprobs': None}, id='run-785ec81d-b8cb-40ac-9f9c-129dd2eda0b1-0', usage_metadata={'input_tokens': 58, 'output_tokens': 14, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [93]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

class BufferWindowMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing BufferWindowMessageHistory with k={k}")

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history, removing any messages beyond
        the last `k` messages.
        """
        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [94]:
chat_map = {}
def get_chat_history(session_id: str, k: int = 4) -> BufferWindowMessageHistory:
    print(f"get_chat_history called with session_id={session_id} and k={k}")
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = BufferWindowMessageHistory(k=k)
    # remove anything beyond the last
    return chat_map[session_id]

In [95]:
pipeline = prompt | model

In [98]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="The number of messages to keep in the history",
            default=4,
        )
    ]
)

In [99]:
pipeline_with_history.invoke(
    {"input": "Hi, my name is Juan"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4


AIMessage(content='Hello Juan! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 34, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEeU0gefil3M0KKrLtUBXo8GPTWbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-15737a81-7c34-4d1d-81a2-29f45a8f4a45-0', usage_metadata={'input_tokens': 34, 'output_tokens': 11, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [100]:
chat_map["id_k4"].messages

[HumanMessage(content='Hi, my name is Juan', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Juan! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 34, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEeU0gefil3M0KKrLtUBXo8GPTWbb', 'finish_reason': 'stop', 'logprobs': None}, id='run-15737a81-7c34-4d1d-81a2-29f45a8f4a45-0', usage_metadata={'input_tokens': 34, 'output_tokens': 11, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [102]:
pipeline_with_history.invoke(
    {"input": "what is my name again?"},
    config={"configurable": {"session_id": "id_k4", "k": 4}}
)

get_chat_history called with session_id=id_k4 and k=4


AIMessage(content='Your name is Juan.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 58, 'total_tokens': 64, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEeWOZpIYVnuxA4jr0U3nbv7GtElD', 'finish_reason': 'stop', 'logprobs': None}, id='run-e202d3b1-5d2d-4b05-910f-1429a112f7bc-0', usage_metadata={'input_tokens': 58, 'output_tokens': 6, 'total_tokens': 64, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [117]:
from langchain_core.messages import SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)


class ConversationSummaryMessageHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)
    llm: ChatOpenAI = Field(default_factory=ChatOpenAI)

    def __init__(self, llm: ChatOpenAI):
        super().__init__(llm=llm)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add messages to the history and summarize them."""
        self.messages.extend(messages)
        
        # Get existing summary or use an empty string if no messages exist
        existing_summary = ""
        if self.messages and len(self.messages) > 0:
            existing_summary = "\n".join([msg.content for msg in self.messages])
        
        # construct the summary chat messages
        summary_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(
                "Given the existing conversation summary and the new messages, "
                "generate a new summary of the conversation. Ensuring to maintain "
                "as much relevant information as possible."
            ),
            HumanMessagePromptTemplate.from_template(
                "Existing conversation summary:\n{existing_summary}\n\n"
                "New messages:\n{messages}"
            )
        ])
        
        # format the messages and invoke the LLM
        new_summary = self.llm.invoke(
            summary_prompt.format_messages(
                existing_summary=existing_summary,
                messages=[x.content for x in messages]
            )
        )
        
        # replace the existing history with a single system summary message
        self.messages = [SystemMessage(content=new_summary.content)]

    def clear(self) -> None:
        """Clear the history."""
        self.messages = []

In [118]:
chat_map = {}
def get_chat_history(session_id: str, llm: ChatOpenAI) -> ConversationSummaryMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = ConversationSummaryMessageHistory(llm=llm)
    # return the chat history
    return chat_map[session_id]

In [119]:
pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="The session ID to use for the chat history",
            default="id_default",
        ),
        ConfigurableFieldSpec(
            id="llm",
            annotation=ChatOpenAI,
            name="LLM",
            description="The LLM to use for the conversation summary",
            default=model,
        )
    ]
)

In [120]:
pipeline_with_history.invoke(
    {"input": "Hi, my name is Juan"},
    config={"session_id": "id_123", "llm": model}
)

AIMessage(content='Hello Juan! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 34, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEf0z6f76bXWyy6FBUfASjJpEOIog', 'finish_reason': 'stop', 'logprobs': None}, id='run-00fa157b-987e-49cb-bbb8-6e6b1ea92389-0', usage_metadata={'input_tokens': 34, 'output_tokens': 11, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [121]:
chat_map["id_123"].messages

[SystemMessage(content='Juan introduced himself and welcomed the assistance offered by the other person in the conversation.', additional_kwargs={}, response_metadata={})]

In [123]:
pipeline_with_history.invoke(
    {"input": "I'm researching the different types of conversational memory."},
    config={"session_id": "id_123", "llm": model}
)

AIMessage(content="That's an interesting topic! Conversational memory can refer to different aspects of memory related to conversations. Some common types of conversational memory include working memory (the temporary storage and manipulation of information during a conversation), episodic memory (memory for specific events or episodes in a conversation), and semantic memory (general knowledge about language and conversation). \n\nIf you have any specific questions or if you need more information on a particular aspect of conversational memory, feel free to ask!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 59, 'total_tokens': 154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BEf3

In [124]:
chat_map["id_123"].messages

[SystemMessage(content='Juan and the other person in the conversation discussed the topic of conversational memory, touching on different aspects such as working memory, episodic memory, and semantic memory. The conversation invited questions about specific aspects of conversational memory for further discussion and exploration.', additional_kwargs={}, response_metadata={})]

In [125]:
pipeline_with_history.invoke(
    {"input": "Explain in details what is an AI agent"},
    config={"session_id": "id_123", "llm": model}
)

AIMessage(content='An AI (Artificial Intelligence) agent is a software program or system that is designed to act intelligently to achieve a certain goal or perform specific tasks. AI agents are capable of perceiving their environment, making decisions, and taking actions to achieve their objectives. These agents can operate autonomously, adapt to changing conditions, and learn from experience to improve their performance over time.\n\nThere are various types of AI agents, each with specific characteristics and capabilities:\n\n1. **Simple Reflex Agents**: These agents make decisions based only on the current percept (input) without considering past perceptions or future consequences. They operate using a simple "if-then" rule-based system.\n\n2. **Model-Based Reflex Agents**: These agents maintain an internal state representing their model of the world, allowing them to make decisions based on both the current percept and the internal world model.\n\n3. **Goal-Based Agents**: These age

In [126]:
chat_map["id_123"].messages

[SystemMessage(content='The conversation delved into the detailed explanation of what an AI agent is, describing it as a software program or system designed to act intelligently to achieve goals. Various types of AI agents were discussed, including simple reflex agents, model-based reflex agents, goal-based agents, utility-based agents, learning agents, logical agents, and software agents. Each type possesses unique characteristics and capabilities in perceiving environments, decision-making, and achieving objectives. The utilization of AI agents in applications like autonomous vehicles, recommendation systems, natural language processing, and robotics was highlighted as enabling automation and intelligent decision-making across diverse domains.', additional_kwargs={}, response_metadata={})]